In [23]:
import pandas as pd
import metadata_parser
import numpy as np
import numpy as np
import time
import math
import urllib
import mimetypes

In [24]:
# read file
path = 'data\1_data_selection/combine_split/'
path_save = 'data\2_data_addition_scrapping\sistem_dodo'

file_name = 'sistem_dodo_b1.csv'
df = pd.read_csv(path+file_name)
df = pd.read_csv(path+file_name)
df.head()

,url
0,http://jb.siyonli.com/sign/l7slug3sg/t/888/164...
1,http://d.lmuda.com/sign/6lh442nyk/t/999/167527...
2,http://jb.siyonli.com/sign/wrwwyvr6g/t/888/164...
3,http://d.lmuda.com/sign/q7ldo2j4n/t/999/167527...
4,http://d.lmuda.com/sign/14cgf47qb/t/999/167527...


In [25]:
# 'https://m-w.com' -> application/x-msdos-program
# 'https://cornell.edu' -> text/html

#function
def guess_type_url(link, strict=True):
    try :
      link_type, _ = mimetypes.guess_type(link)
      if link_type is None and strict:
          u = urllib.request.urlopen(link)
          link_type = u.headers.get_content_type()
    except Exception as e: 
      link_type = "[ERROR] " + str(e)
    return link_type

In [26]:
#function
def get_metadata_s(url,meta_name):
    content = []
    url_file_type = guess_type_url(url)
    if url_file_type in ['application/x-msdos-program', 'text/html']:
      try:
          page = metadata_parser.MetadataParser(url)
      except:
          return ['error']*len(meta_name)+[url_file_type]
      for m in meta_name:
          try:
              content.append(page.get_metadata(m, strategy=['meta','page','og','dc']))
          except:
              content.append('error')
      content+=[url_file_type]
    else :
      content = ['error']*len(meta_name)+[url_file_type]
    return np.array(content)  

In [27]:
#function
def cron_meta_data_s(df,timeout_m,meta_name):
    """
        df : dataframe
        timeout_m : time limit of the process in minute
        meta_name : meta tag name, column name
    """

    timeout = timeout_m * 60
    timeout_start = time.time()
    batch_size = 5
    total_batch = math.ceil(len(df)/batch_size)
    index = 1
    col_index = df.columns.get_indexer(meta_name+['url_file_type'])
    while (time.time() < timeout_start + timeout) and index <= total_batch:
        if index == total_batch:
            df.iloc[(index-1)*batch_size:-1,col_index] = np.array([np.array(a) for a in zip(*df['url'][(index-1)*batch_size:-1].apply(lambda x: get_metadata_s(x,meta_name)))]).T
        else:
            df.iloc[(index-1)*batch_size:index*batch_size,col_index] = np.array([np.array(a) for a in zip(*df['url'][(index-1)*batch_size:index*batch_size].apply(lambda x: get_metadata_s(x,meta_name)))]).T
        index += 1
    print("last index : {} ======================================================".format(index))

In [28]:
meta_tag_name = ['description', 'title']

In [11]:
#satuan
# url ='https://functional.events.data.microsoft.com/OneCollector/1.0/'
# # url = 'https://'+url
# meta_tag_name = ['description', 'title']
# hasil = get_metadata_s(url,meta_tag_name)
# print('==================================================')
# print('==================================================')
# print(hasil)

In [29]:
# getting data -> continue from old file
start_index = 0
df_continue = df.iloc[start_index:]
df_continue.head()

,url
0,http://jb.siyonli.com/sign/l7slug3sg/t/888/164...
1,http://d.lmuda.com/sign/6lh442nyk/t/999/167527...
2,http://jb.siyonli.com/sign/wrwwyvr6g/t/888/164...
3,http://d.lmuda.com/sign/q7ldo2j4n/t/999/167527...
4,http://d.lmuda.com/sign/14cgf47qb/t/999/167527...


In [30]:
df[meta_tag_name+['url_file_type']] = ''
cron_meta_data_s(df_continue,10,meta_tag_name)

# if file_name[3:5] == '_c' :
#     file_name_new = file_name[:5] + str(int(file_name[5:7])+1).zfill(2)  + file_name[7:]
# else:
#     file_name_new = file_name[:4] + 'c01_' + file_name[4:]




c:\Users\binta\anaconda3\lib\site-packages\metadata_parser\__init__.py:1331: FutureWarning: `search_head_only` was not provided and defaulting to `True` Future versions will default to `False`.
  warn_future(
c:\Users\binta\anaconda3\lib\site-packages\metadata_parser\__init__.py:979: FutureWarning: `get_metadata` returns a string and is being deprecatedin favor of `get_metadatas` which returns a list.
  warn_future(
c:\Users\binta\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-27-bf59b9a28976>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

last index : 280 ======================================================


In [31]:
file_name_new = 'p2_sistem_dodo_b1_c00.csv'
df.to_csv(path_save+file_name_new)

In [22]:
df_temp = df.copy()